In [7]:
import pandas as pd
import os

"""
Como ira organizado el DataFrame

Games | ReleaseDate | CharactersPath | Developer | 2D | 3D | TagTeam | AirDashers | Anime | WeaponBased | Fast-paced | Footsies | SlowPaced | Installbased | CrossOver
Nombre | Fecha de Lanzamiento | Ruta_en_dir_data (del archivo con los links de personajes) | Desarrollador | y el resto con una X mayuscula si tiene esa caracteristica o un espacio si no la tiene
"""


df = pd.DataFrame({"Games":[],
                   "ReleaseDate":[],
                   "CharactersPath":[],
                   "Developer":[], 
                   "2D":[], 
                   "3D":[], 
                   "TagTeam":[], 
                   "AirDashers":[], 
                   "Anime":[], 
                   "WeaponBased":[], 
                   "Fast-paced":[],
                   "Footsies":[],
                   "SlowPaced":[],
                   "Installbased":[],
                   "CrossOver":[]
                   })



#funcion que crea un archivo csv con el nombre del juego y la link de discord de la imagen de los personajes.

def CreateCharacterFile(GameName)-> None:
    file_name =  f"Data/{GameName}.csv"

    if os.path.exists(file_name):
        return None

    with open(file_name, "w") as f:

        # First row is the header, put CharacterName and ImageLink
        f.write("CharacterName,ImageLink\n")
        f.close()



IndentationError: expected an indented block after 'if' statement on line 36 (1139728633.py, line 37)

In [5]:

"""
Agregar un nuevo juego por medio de los archivos tabulados
Los archivos tabulados deben de estar en la carpeta TabulatedData
"""

with open(f"TabulatedData/GameName.csv", "r") as f:
    for line in f:
        GameName, ReleaseDate = line.split(",")
        GameName = GameName.strip()
        ReleaseDate = ReleaseDate.strip()
        CreateCharacterFile(GameName)
        df = df.add({"Games":GameName,
                        "ReleaseDate":ReleaseDate,
                        "CharactersPath":f"Data/{GameName}.csv",
                        "Developer":"", 
                        "2D":"", 
                        "3D":"", 
                        "TagTeam":"", 
                        "AirDashers":"", 
                        "Anime":"", 
                        "WeaponBased":"", 
                        "Fast-paced":"",
                        "Footsies":"",
                        "SlowPaced":"",
                        "Installbased":"",
                        "CrossOver":""
                        })